## Requirements

In [1]:
from distutils.filelist import findall
import pandas as pd
from pmaw import PushshiftAPI
import datetime as dt
import os
import requests
import re
from bs4 import BeautifulSoup as bs
import spacy_udpipe

## Retreiving posts

#### Load Pushshift

In [2]:
api = PushshiftAPI()

#### Search parameters


##### Specify time frame

In [3]:
start_time = int(dt.datetime.timestamp(dt.datetime.strptime('2011-02-14 00:00:00', '%Y-%m-%d %H:%M:%S'))) #time EarthPorn was created
end_time = int(dt.datetime.timestamp(dt.datetime.strptime('2022-08-01 00:00:00', '%Y-%m-%d %H:%M:%S')))
current_time = int(dt.datetime.timestamp(dt.datetime.now()))

# Create string specifying time frame that can be used for file name when saving data as csv
search_time = '20110214-20220816' 

##### Specify subreddit and search limit

In [4]:
# Set subreddit and limit
subreddit = 'EarthPorn'
limit = None

#### Query posts from pushshift using search_submissions with default parameters

Default parameters:  
max_ids_per_request = 500 (max)  
max_results_per_request = 100 (max)  
mem_safe = False -> stores responses in cache during operation if True  
safe_exit = False -> will safely exit if interupted by storing current responses and requests in the cache if True  
cache_dir -> path to cache responses in when mem_safe or safe_exit is enabled  

In [5]:
posts = api.search_submissions(subreddit=subreddit, limit=limit, before=current_time, after=start_time)
print(f'Retrieved {len(posts)} posts from Pushshift')

Retrieved 20499 posts from Pushshift


#### Create data frame for posts

In [6]:
post_list = [post for post in posts]
posts_df = pd.DataFrame(post_list)

Preview sample of posts data

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 25)
print(posts_df)

       subreddit selftext author_fullname  gilded  \
0      EarthPorn              t2_vt116poy       0   
1      EarthPorn               t2_5ct5sg1       0   
2      EarthPorn              t2_6036b6ml       0   
3      EarthPorn             t2_6c9gdidyj       0   
4      EarthPorn              t2_8y90xn4c       0   
...          ...      ...             ...     ...   
20494  EarthPorn              t2_dljmojix       0   
20495  EarthPorn                 t2_o9xid       0   
20496  EarthPorn             t2_45z17uz3e       0   
20497  EarthPorn                 t2_njvc8       0   
20498  EarthPorn              t2_orgk52et       0   

                                                   title link_flair_richtext  \
0                   Just another day on the Oregon coast                  []   
1                          South Beach, Friday Harbor WA                  []   
2                                          Zion NP, Utah                  []   
3                              Duck Lake, C

#### Clean up data frame

##### Get list of all column names

In [8]:
posts_df.columns

Index(['subreddit', 'selftext', 'author_fullname', 'gilded', 'title',
       'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls',
       'link_flair_css_class', 'thumbnail_height', 'top_awarded_type',
       'hide_score', 'quarantine', 'link_flair_text_color', 'upvote_ratio',
       'author_flair_background_color', 'subreddit_type',
       'total_awards_received', 'media_embed', 'thumbnail_width',
       'author_flair_template_id', 'is_original_content', 'secure_media',
       'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed',
       'link_flair_text', 'score', 'is_created_from_ads_ui', 'author_premium',
       'thumbnail', 'edited', 'author_flair_css_class',
       'author_flair_richtext', 'gildings', 'post_hint', 'content_categories',
       'is_self', 'link_flair_type', 'wls', 'removed_by_category',
       'author_flair_type', 'domain', 'allow_live_comments', 'suggested_sort',
       'url_overridden_by_dest', 'view_count', 'archived', 'no_follow',


##### Remove unwanted columns

In [9]:
posts_df.drop(columns = ['author_flair_css_class', 'author_flair_text'], inplace = True)

#posts_df.drop(columns = ['author_flair_css_class', 'author_flair_text', 'gilded', 'mod_reports', 'user_reports', 'brand_safe', 'contest_mode', 'spoiler', 'suggested_sort', 'author_flair_richtext', 'author_flair_type', 'can_mod_post', 'link_flair_richtext', 'link_flair_text_color', 'link_flair_type', 'rte_mode', 'subreddit_type', 'thumbnail_height', 'thumbnail_width', 'author_flair_background_color', 'author_flair_text_color', 'author_patreon_flair', 'gildings', 'is_robot_indexable', 'link_flair_background_color', 'send_replies', 'no_follow', 'updated_utc', 'all_awardings', 'allow_live_comments', 'author_premium', 'awarders', 'total_awards_received', 'treatment_tags', 'is_created_from_ads_ui', 'parent_whitelist_status', 'pwls', 'url_overridden_by_dest', 'whitelist_status', 'wls', 'removed_by_category', 'author_is_blocked', 'approved_at_utc', 'banned_at_utc', 'steward_reports', 'removed_by', 'poll_data', 'top_awarded_type', 'retrieved_on'], inplace = True)

##### Change column names and reorder columns

1. Create dictionary - 'old name' : 'new name'

In [10]:
column_names = {'id' : 'PostID',
                'subreddit' : 'Subreddit',
                'subreddit_id' : 'SubredditID',
                'created_utc' : 'PostTime',
                'title' : 'PostTitle',
                'author' : 'Username',
                'author_created_utc' : 'UserCreatedTime',
                'author_fullname' : 'AuthorName', 
                'domain' : 'ImageDomain',
                'full_link' : 'Link',
                'is_self' : 'IsTextPost',
                'media_embed' : 'EmbeddedMedia',
                'secure_media_embed' : 'SecureEmbeddedMedia',
                'num_comments' : 'CommentNumber', 
                'over_18' : 'NSFW',
                'permalink' : 'Permalink', 
                'score' : 'Upvotes', 
                'selftext' : 'PostText', 
                'thumbnail' : 'Thumbnail',
                'url' : 'ImageURL',
                'media' : 'Media',
                'secure_media' : 'SecureMedia',
                'stickied' : 'Stickied',
                'locked' : 'CommentsLocked',
                'post_hint' : 'PostHint',
                'preview' : 'Preview',
                'is_crosspostable' : 'IsCrosspostable',
                'is_reddit_media_domain' : 'IsRedditMediaDomain',
                'is_video' : 'IsVideo',
                'num_crossposts' : 'CrosspostsNumber', 
                'pinned' : 'Pinned',
                'crosspost_parent' : 'CrosspostParent',
                'crosspost_parent_list' : 'CrosspostParentList',
                'is_meta' : 'IsMeta',
                'is_original_content' : 'IsOriginal',
                'media_only' : 'OnlyMedia', 
                'subreddit_subscribers' : 'SubRedditSubscribers',
                'media_metadata' : 'MediaMetadata', 
                'upvote_ratio' : 'UpvoteRatio', 
                'gallery_data' : 'GalleryData', 
                'is_gallery' : 'IsGallery', 
                'author_cakeday' : 'AuthorBirthdate',
                'edited' : 'Edited', 
                'view_count' : 'ViewCount', 
                'author_id' : 'AuthorID',
                'og_description' : 'OGDescription',
                'og_title' : 'OGTitle'}

2. Rename columns using dictionary

In [11]:
posts_tidy_df = posts_df.rename(columns = column_names)
# Check to see if columns have been renamed
posts_tidy_df.columns 

Index(['Subreddit', 'PostText', 'AuthorName', 'gilded', 'PostTitle',
       'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls',
       'link_flair_css_class', 'thumbnail_height', 'top_awarded_type',
       'hide_score', 'quarantine', 'link_flair_text_color', 'UpvoteRatio',
       'author_flair_background_color', 'subreddit_type',
       'total_awards_received', 'EmbeddedMedia', 'thumbnail_width',
       'author_flair_template_id', 'IsOriginal', 'SecureMedia',
       'IsRedditMediaDomain', 'IsMeta', 'category', 'SecureEmbeddedMedia',
       'link_flair_text', 'Upvotes', 'is_created_from_ads_ui',
       'author_premium', 'Thumbnail', 'Edited', 'author_flair_richtext',
       'gildings', 'PostHint', 'content_categories', 'IsTextPost',
       'link_flair_type', 'wls', 'removed_by_category', 'author_flair_type',
       'ImageDomain', 'allow_live_comments', 'suggested_sort',
       'url_overridden_by_dest', 'ViewCount', 'archived', 'no_follow',
       'IsCrosspostable', 'Pinn

3. Reorder columns

In [12]:
#posts_tidy_df = posts_tidy_df[['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'PostTime', 'Username', 'Upvotes', 'CommentNumber', 'ImageDomain', 'ImageURL', 'UserCreatedTime', 'AuthorName', 'Permalink', 'Link', 'IsTextPost', 'PostText', 'EmbeddedMedia', 'Thumbnail', 'NSFW']]

posts_tidy_df = posts_tidy_df[['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'PostTime', 'Username', 'Upvotes', 'CommentNumber', 'ImageDomain', 'ImageURL', 'AuthorName', 'Permalink', 'IsTextPost', 'PostText', 'EmbeddedMedia', 'Thumbnail', 'NSFW']]
                                       

#posts_reordered_df = posts_renamed_df[['Subreddit', 'SubredditID', 'PostTitle', 'PostID', 'PostTime', 'Username', 'ViewCount', 'Upvotes', 'UpvoteRatio', 'CommentNumber', 'Edited', 'OGDescription', 'OGTitle', 'ImageDomain', 'ImageURL', 'Permalink', 'Link', 'IsTextPost', 'PostText', 'UserCreatedTime', 'AuthorID', 'AuthorName', 'AuthorBirthdate', 'IsVideo', 'IsMeta', 'IsOriginal', 'IsGallery', 'GalleryData', 'IsRedditMediaDomain', 'IsCrosspostable', 'CrosspostsNumber', 'CrosspostParent', 'CrosspostParentList', 'SubRedditSubscribers', 'OnlyMedia', 'MediaMetadata', 'EmbeddedMedia', 'SecureEmbeddedMedia', 'Media', 'SecureMedia', 'Thumbnail', 'Stickied', 'Pinned', 'PostHint', 'Preview', 'CommentsLocked', 'NSFW']]

Convert time stamp from UNIX to UTC

In [13]:
posts_tidy_df['PostTime'] = pd.to_datetime(posts_tidy_df['PostTime'], utc=True, unit='s')

#### Fix image URLS

In [14]:
posts_tidy_df = posts_tidy_df.reindex(columns = posts_tidy_df.columns.tolist() + ['NewURL']) #create column for fixed urls

for index, row in posts_tidy_df.iterrows():
    if row['ImageDomain'] == 'flickr.com':
        print(row['ImageURL'])
        r = requests.get(row['ImageURL'])
        soup = bs(r.content)
        images = re.findall(r'(https:\/\/live\.staticflickr\.com\/[0-9][0-9][0-9][0-9]\/[a-zA-Z0-9_]+\.(?:png|jpg|jpeg|gif|png|svg))', str(soup))
        for image in images:
            image_url = image
            break
        print(image_url)
        if image_url != None:
            posts_tidy_df.at[index, 'NewURL'] = image_url
    elif row['ImageDomain'] == 'imgur.com':
        posts_tidy_df.at[index, 'NewURL'] = re.sub(r'http://imgur.com', 'http://i.imgur.com', row['ImageURL']) + '.jpg'
    elif row['ImageDomain'] == 'i.imgur.com':
        posts_tidy_df.at[index, 'NewURL'] = row['ImageURL']
    elif row['ImageDomain'] == 'i.redd.it':
        posts_tidy_df.at[index, 'NewURL'] = row['ImageURL']
    else:
        continue

In [15]:
posts_tidy_df = posts_tidy_df.dropna()

## Save data frame and images

#### View data frame

In [16]:
pd.set_option('display.max_colwidth', None)
posts_tidy_df

,Subreddit,SubredditID,PostTitle,PostID,PostTime,Username,Upvotes,CommentNumber,ImageDomain,ImageURL,AuthorName,Permalink,IsTextPost,PostText,EmbeddedMedia,Thumbnail,NSFW,NewURL
0,EarthPorn,t5_2sbq3,Just another day on the Oregon coast,11krpjh,2023-03-07 06:37:47+00:00,IndignantFigment,1,1,i.imgur.com,https://i.imgur.com/R1eKfzh.jpg,t2_vt116poy,/r/EarthPorn/comments/11krpjh/just_another_day_on_the_oregon_coast/,False,,{},https://b.thumbs.redditmedia.com/wx0v-oX-UiEnYmM9Qv6LE8HY_M4tJHSkd71QdJr6Hts.jpg,False,https://i.imgur.com/R1eKfzh.jpg
1,EarthPorn,t5_2sbq3,"South Beach, Friday Harbor WA",11krf0x,2023-03-07 06:22:03+00:00,xPapaBojanglesx,1,1,i.imgur.com,https://i.imgur.com/fnMTjSd.jpg,t2_5ct5sg1,/r/EarthPorn/comments/11krf0x/south_beach_friday_harbor_wa/,False,,{},https://b.thumbs.redditmedia.com/8CdkH1o9dbXrV75Y2-0xdqEVjRWmAeqwsjv0DfmneAQ.jpg,False,https://i.imgur.com/fnMTjSd.jpg
2,EarthPorn,t5_2sbq3,"Zion NP, Utah",11kqmi6,2023-03-07 05:42:12+00:00,aakline800,1,1,i.redd.it,https://i.redd.it/hf4cebj9qama1.jpg,t2_6036b6ml,/r/EarthPorn/comments/11kqmi6/zion_np_utah/,False,,{},https://b.thumbs.redditmedia.com/k9zkn_aqBL4gQL0BOuOiTjpRXBr80C1nS9qWCSgqnmg.jpg,False,https://i.redd.it/hf4cebj9qama1.jpg
3,EarthPorn,t5_2sbq3,"Duck Lake, Ca [4032x3024]",11kqcoh,2023-03-07 05:28:45+00:00,DAVELASOUL619,1,1,i.redd.it,https://i.redd.it/f3rgl2xunama1.jpg,t2_6c9gdidyj,/r/EarthPorn/comments/11kqcoh/duck_lake_ca_4032x3024/,False,,{},https://b.thumbs.redditmedia.com/RxnbINoU8Br9S_Grvgtb7cRAtdkEWOpvDq1br0940ZM.jpg,False,https://i.redd.it/f3rgl2xunama1.jpg
4,EarthPorn,t5_2sbq3,"Yoho National Park, British Columbia [OC] (3024x4032)",11kpqc0,2023-03-07 04:59:33+00:00,KoalaLumpur22,1,0,i.redd.it,https://i.redd.it/7u8e5wu419ma1.jpg,t2_8y90xn4c,/r/EarthPorn/comments/11kpqc0/yoho_national_park_british_columbia_oc_3024x4032/,False,,{},https://b.thumbs.redditmedia.com/ShI0tJ0A35LiYUlNXJ-K4SaOSwgfTp0S33NXVH4r7KE.jpg,False,https://i.redd.it/7u8e5wu419ma1.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20494,EarthPorn,t5_2sbq3,"Glencoe, Scotland 🏴󠁧󠁢󠁳󠁣󠁴󠁿 🇬🇧",11k10b3,2023-03-06 14:55:47+00:00,Shifty2805,1,1,i.redd.it,https://i.redd.it/ywoevt84c6ma1.jpg,t2_dljmojix,/r/EarthPorn/comments/11k10b3/glencoe_scotland/,False,,{},https://b.thumbs.redditmedia.com/aklVVbmFteoaEgMMEgXveHwUhHqnAEAjntconZxbXWs.jpg,False,https://i.redd.it/ywoevt84c6ma1.jpg
20495,EarthPorn,t5_2sbq3,"Laguna Paron in Huascaran National Park, Peru (OC) [3000x4000]",11k0yxu,2023-03-06 14:54:13+00:00,MozillaFirefucks,1,0,i.imgur.com,https://i.imgur.com/dYOKgIx.jpg,t2_o9xid,/r/EarthPorn/comments/11k0yxu/laguna_paron_in_huascaran_national_park_peru_oc/,False,,{},https://b.thumbs.redditmedia.com/50LTksZtE2FYisigf3QVQBbvoZcQgjBmTZLPbxuFBDk.jpg,False,https://i.imgur.com/dYOKgIx.jpg
20496,EarthPorn,t5_2sbq3,Phone photo s21,11k0whv,2023-03-06 14:51:21+00:00,Hemligkille94,1,1,i.redd.it,https://i.redd.it/582ik4zbb6ma1.jpg,t2_45z17uz3e,/r/EarthPorn/comments/11k0whv/phone_photo_s21/,False,,{},nsfw,True,https://i.redd.it/582ik4zbb6ma1.jpg
20497,EarthPorn,t5_2sbq3,"Maui, Hawaii [3024 x 4032]",11k0wb6,2023-03-06 14:51:08+00:00,FunkHavoc,1,1,i.redd.it,https://i.redd.it/m00bg4z9b6ma1.jpg,t2_njvc8,/r/EarthPorn/comments/11k0wb6/maui_hawaii_3024_x_4032/,False,,{},https://b.thumbs.redditmedia.com/Hqp5g-l0J9laFmiMbtx-aP3yWiPS_yT1CUPF4Jj0bQk.jpg,False,https://i.redd.it/m00bg4z9b6ma1.jpg


#### Save data frame as CSV

In [17]:
filename = '/Users/clarasofiechristiansen/Documents/Clara/DTU/Data_Fagprojekt/' + subreddit + '_' + search_time + '.csv'
posts_tidy_df.to_csv(filename, header=True, index=False, columns=list(posts_tidy_df.axes[1]))

#### Save images from URLs

In [18]:
#Save images from data frame URL column
root_folder = '/Users/clarasofiechristiansen/Documents/Clara/DTU/Data_Fagprojekt/'

def download(row):
   filename = root_folder + subreddit + '_' + row['PostID'] + '.jpg'

   # create folder if it doesn't exist
   os.makedirs(os.path.dirname(filename), exist_ok = True)

   url = row.NewURL

   print(f"Downloading {url} to {filename}")
   r = requests.get(url, allow_redirects=True)
   with open(filename, 'wb') as f:
       f.write(r.content)

try:
    posts_tidy_df.apply(download, axis=1)
except:
    pass

## Text cleaning and annotating features

> this might be useful later on to create a list of features mentioned in the text for each picture

#### Remove brackets and other characters

In [ ]:
posts_clean_df = posts_tidy_df.rename(columns = column_names)
posts_clean_df['PostTitle'].replace(to_replace="\[(.*?)\]", value="", regex=True, inplace=True) 
posts_clean_df['PostTitle'].replace(to_replace="\(\d*?\s*[\u00D7?x?]\s*\d*?\)", value="", regex=True, inplace=True)
posts_clean_df['PostTitle'].replace(to_replace="\(", value="", regex=True, inplace=True)
posts_clean_df['PostTitle'].replace(to_replace="\)", value="", regex=True, inplace=True)
posts_clean_df['PostTitle'].replace(to_replace="-", value="", regex=True, inplace=True)

#### Load NLP model

In [ ]:
spacy_udpipe.download("en")
nlp = spacy_udpipe.load("en")

Downloaded pre-trained UDPipe model for 'en' language


##### Create new data frame for annotations

In [ ]:
column_names = ['Sentence', 'Text ID', 'IDX', 'Text', 'Lemma', 'POS', 'Form', 'Dependency', 'Sentiment'] 
posts_annotated_df = pd.DataFrame(columns=column_names)

#### Create empty lists to store token values in

In [ ]:
sent = []
i = []
idx = []
word = []
lemma = []
pos = []
tag = []
dep = []
sentiment = []
form = []

#### Tokenize post titles

In [ ]:
for index, row in posts_clean_df.iterrows():
    text = row['PostTitle']
    doc = nlp(text)
    for token in doc:
        #print('Sentence:' + token.sent)
        sent.append(token.sent)
        i.append(token.i)
        idx.append(token.idx)
        word.append(token.text)
        lemma.append(token.lemma_)
        pos.append(token.pos_)
        form.append(token.morph.get("VerbForm"))
        tag.append(token.tag_)
        dep.append(token.dep_)
        sentiment.append(token.sentiment)
       

#### Add token annotations to data frame

In [ ]:
posts_annotated_df['Sentence'] = sent
posts_annotated_df['Text ID'] = i
posts_annotated_df['Text'] = word
posts_annotated_df['Lemma'] = lemma
posts_annotated_df['POS'] = pos
posts_annotated_df['VerbForm'] = form
posts_annotated_df['Dependency'] = dep
posts_annotated_df['IDX'] = idx
posts_annotated_df['Sentiment'] = sentiment
posts_annotated_df['VerbForm'] = posts_annotated_df['VerbForm'].str[0]

print(posts_annotated_df)

                                                                                                                     Sentence  \
0      (Red, Tarn, and, Catstye, Cam, as, seen, from, Striding, Edge, at, the, base, of, Helvellyn, ,, Lake, District, ,, UK)   
1      (Red, Tarn, and, Catstye, Cam, as, seen, from, Striding, Edge, at, the, base, of, Helvellyn, ,, Lake, District, ,, UK)   
2      (Red, Tarn, and, Catstye, Cam, as, seen, from, Striding, Edge, at, the, base, of, Helvellyn, ,, Lake, District, ,, UK)   
3      (Red, Tarn, and, Catstye, Cam, as, seen, from, Striding, Edge, at, the, base, of, Helvellyn, ,, Lake, District, ,, UK)   
4      (Red, Tarn, and, Catstye, Cam, as, seen, from, Striding, Edge, at, the, base, of, Helvellyn, ,, Lake, District, ,, UK)   
...                                                                                                                       ...   
72114                                                              (Cold, winter, river, below, M

##### Save annotations as CSV

In [ ]:
filename = '/Users/clarasofiechristiansen/Documents/Clara/DTU/Data_Fagprojekt/' + subreddit + '_' + search_time + '.csv'
posts_annotated_df.to_csv(filename, header=True, index=False, columns=list(posts_annotated_df.axes[1]))

### Find features

#### Filter for nouns

In [ ]:
nouns_df = posts_annotated_df[posts_annotated_df['POS'].str.contains("NOUN|PROPNOUN")]
features_df = nouns_df[nouns_df['Dependency'].str.contains('ROOT')]

#### Create list with features

In [ ]:
features_list = features_df['Text'].tolist()

#### Create data frame with features and subreddit name

In [ ]:
CES_features = pd.DataFrame()
CES_features['Features'] = features_list
CES_features['Subreddit'] = subreddit

#### Save features as CSV

In [ ]:
#All_CES_features_updated.to_csv(features_filename, header=True, index=False, columns=list(All_CES_features_updated.axes[1]))

NameError: name 'All_CES_features_updated' is not defined

In [ ]:
filename = '/Users/clarasofiechristiansen/Documents/Clara/DTU/Data_Fagprojekt/CES_features.csv'
CES_features.to_csv(filename, header=True, index=False, columns=list(CES_features.axes[1]))